In [1]:
!pip install facenet-pytorch
!pip install mmcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 547 kB 5.0 MB/s 
     |████████████████████████████████| 190 kB 57.1 MB/s 
  Created wheel for mmcv: filename=mmcv-1.5.3-py2.py3-none-any.whl size=834599 sha256=b9a4ed9c86b041c74555a477c75ffcaa592eb7c54e6de479d975eeea0dbb49c5
  Stored in directory: /root/.cache/pip/wheels/45/bc/06/d92ae8de9d1acee0e967c73f918664c9b43bb16e0089a37484
Successfully built mmcv


In [2]:
!git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch

Cloning into 'facenet_pytorch'...
remote: Enumerating objects: 1267, done.
remote: Total 1267 (delta 0), reused 0 (delta 0), pack-reused 1267
Receiving objects: 100% (1267/1267), 22.88 MiB | 37.25 MiB/s, done.
Resolving deltas: 100% (619/619), done.


In [3]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display
from ipywidgets import Video
import moviepy.editor as mp


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
mtcnn = MTCNN(keep_all=True, device=device)

In [6]:
def main(v_name):
  out_clean = f'{v_name.split("/")[-1].split(".")[0]}_tracked'
  out_mp4 = f'{out_clean}.mp4'
  out_gif = f'{out_clean}.gif'
  video = mmcv.VideoReader(v_name)
  frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

  frames_tracked = []
  for i, frame in enumerate(frames):
      print('\rTracking frame: {}'.format(i + 1), end='')
      
      # Detect faces
      boxes, _ = mtcnn.detect(frame)
      
      # Draw faces
      frame_draw = frame.copy()
      draw = ImageDraw.Draw(frame_draw)
      if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
      # Add to frame list
      frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
  print('\nDone')
  frames_tracked[0].save(out_gif, save_all=True,optimize=False, append_images=frames_tracked[1:], loop=0)
  clip = mp.VideoFileClip(f'{out_clean}.gif')
  clip.write_videofile(f'{out_mp4}')
  print(f'{out_mp4} saved')
  return out_mp4

# example 1

In [7]:
v_name = 'v1.mp4'
out_file = main(v_name)
Video.from_file(out_file, play=True)

Tracking frame: 241
Done
[MoviePy] >>>> Building video v1_tracked.mp4
[MoviePy] Writing video v1_tracked.mp4


100%|█████████▉| 241/242 [00:10<00:00, 22.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: v1_tracked.mp4 



Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xc9H\xadmdat\x00\x…

v1_tracked.mp4 saved


#exmaple 2

In [8]:
v_name = 'v2.mp4'
out_file = main(v_name)
Video.from_file(out_file, play=True)

Tracking frame: 192
Done
[MoviePy] >>>> Building video v2_tracked.mp4
[MoviePy] Writing video v2_tracked.mp4


 99%|█████████▉| 192/193 [00:05<00:00, 34.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: v2_tracked.mp4 

v2_tracked.mp4 saved


Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00v\x94\xa3mdat\x00\x…